In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
import matplotlib.pyplot as plt
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
def process_log(file, sid):
  """
  From the log file extract the significant parts in different dataframes:
  -subject: personal values of the subject
  -nasa: nasa questionnaire compiled by the subject, reformatted in a new table
  -final_pos: final configuration and correctness for each puzzle
  -moves: list of moves performed by the subject
  """

  entry = {}

  #subject info
  entry['subject'] = pd.read_csv(file, sep = ';', header= None, encoding='iso-8859-1', nrows=4)
  
  df = pd.read_csv(file,sep=';', header=4,skipfooter = 1,encoding='iso-8859-1')
  df['sid'] = sid
  #nasa questionnaire
  nasa=['mental demand','performance','effort','frustration']
  nasa_values= df[df['time'].isin(nasa)]
  entry['nasa'] =pd.pivot_table(nasa_values[['tangram nr', 'time', 'item']], values='item', index=['tangram nr'],
                  columns=['time'])
  
  #final configuration
  entry['final_pos'] = df[df['time']=='solution']

  #moves list
  df_moves= df[~df['time'].isin(nasa)]
  entry['moves'] = df_moves.drop(df_moves[df_moves['time'] == 'solution'].index)


  return entry

def create_moves_dataframe(logs):
  """
  creates a new dataframe which records the moves
  a "move" merges all the intermediate movements and rotations of a single piece
  and ends when a new piece is selected. 
  The values stored are the initial ('from') and final ('position') coordinates
  including x,y and rotation value
  """
  
  # TODO the "from" value does not really store the original position but the position after the first "movement"
  # TODO might require adding the step-value

  out =[]
  moves_df = pd.concat([d['moves'] for d in logs])
  moves_df = moves_df.astype({'time': float})
  moves_df = moves_df[['sid','tangram nr','item','action','position','time']]
  moves_df=  moves_df.loc[moves_df['action']!='moved to correct location']

  last_moves =moves_df.loc[(moves_df['action'].shift(-1) != moves_df['action']) 
                           | (moves_df['tangram nr'].shift(-1) != moves_df['tangram nr'])
                           |(moves_df['item'].shift(-1) != moves_df['item'])
                           ]


 
  return last_moves.reset_index(drop=True)

def create_solution_dataframe(logs):
  sol_df = pd.concat([d['final_pos'] for d in logs])
  sol_df=sol_df.rename(columns = {'item':'solved','action':'config'})
  sol_df=sol_df.astype({'solved':'int'})
  return sol_df[['sid','tangram nr', 'solved', 'config']].reset_index(drop =True)

def create_subjects_dataframe(logs):
  #subjects_df = pd.DataFrame(columns= ["age","gender", "exp", "studies"])
  #for l in logs:
  #  row = l['subject'].values
  #  subjects_df = pd.concat(subjects_df,pd.DataFrame(l['subject'].values)[1].T)
  subjects_df = pd.concat([pd.DataFrame(d['subject'].values)[1] for d in logs], axis=1).T.reset_index(drop=True)
  subjects_df = subjects_df.rename(columns ={0:'age',1:'gender',2: 'experience', 3:'background'})
  return subjects_df



In [3]:
def baseround(x, base=5):
    return base * round(x/base)

solution_limits = {1:[(-260,120),(-120,140)],2:[(-280,0),(-105,185)],3:[(-320,60),(-140,140)],4: [(-280,0),(-200,300)]}

def get_grid_value(x,y,n,tgn):
  """
  Given turtle coordinates as input, returns the grid value of the tan in a nxn grid around the puzzle
  :param x: x value in turtle window
  :type x: int
  :param y: y value in turtle window
  :type y: int
  :param n: size of the grid
  :type n: int
  :param tgn: tangram number
  :type tgn: int
  
  :return: grid value
  :rtype: int
  """
  xrange = solution_limits.get(tgn)[0]
  yrange = solution_limits.get(tgn)[1]
  xstep=(xrange[1]-xrange[0])/n
  ystep=(yrange[1]-yrange[0])/n
  if (x not in range(xrange[0],xrange[1]+1) or y not in range(yrange[0],yrange[1]+1)):
    return -1
  xgrid = (x-xrange[0]) // xstep

  ygrid = (y-yrange[0]) // ystep 

  return ygrid*n + xgrid

unused_grids = {1:[0],2:[0,15,20],3:[0],4:[0,1,4,5,6,15,16,20,21,22]}

def remove_symm(i,r):

    if i == 'square':
    
        return float(r % 90)
    elif i == 'parallelogram':

        return float(r % 180)
    else:
        return r

def create_grid_step_df(df,gridsize):
    step_df= df[df["action"]!='turned 45 degrees right']
    pos = step_df['position'].tolist()
    pos=[eval(x) for x in pos]
    
    step_df[['x','y','rot']] = pd.DataFrame(pos, index=step_df.index)
    step_df = step_df.drop(['position','time','action'], axis=1)
    
    
    ### here
    print(step_df.item.unique())
    step_df['rot'] = step_df.apply(lambda row: remove_symm(row['item'],row['rot']), axis=1)

    
    
    diff_logs = [102957,104401,102644,100444,92356,93250]
    step_df['y'] = step_df.apply(lambda x: x['y']-70 if (x['tangram nr']==4 and x['sid'] in diff_logs) else x['y'], axis=1)
    #step_df['y'] = step_df.apply(lambda x: x['y']-70 if x['sid'] in diff_logs else x['y'], axis=1)

    step_df['grid_val']=step_df.apply(lambda x: get_grid_value(x['x'],x['y'],gridsize,x['tangram nr']),axis=1)
    
    step_df =step_df.loc[(step_df['item'].shift(-1) != step_df['item']) 
                          | (step_df['tangram nr'].shift(-1) != step_df['tangram nr'])
                          ].reset_index(drop=True)
    
    action_df = step_df[step_df['grid_val']!=-1]
    action_df = action_df[~((action_df['tangram nr']== 1) & (action_df['grid_val'].isin(unused_grids.get(1))))]
    action_df = action_df[~((action_df['tangram nr']== 2) & (action_df['grid_val'].isin(unused_grids.get(2))))]
    action_df = action_df[~((action_df['tangram nr']== 3) & (action_df['grid_val'].isin(unused_grids.get(3))))]
    action_df = action_df[~((action_df['tangram nr']== 4) & (action_df['grid_val'].isin(unused_grids.get(4))))]
    action_df =action_df.loc[(action_df['item'].shift(-1) != action_df['item']) 
                          | (action_df['tangram nr'].shift(-1) != action_df['tangram nr'])
                          ]

    action_df['step'] = action_df.groupby((action_df['tangram nr'] != action_df['tangram nr'].shift(1)).cumsum()).cumcount()+1
    
    step_df = pd.concat([step_df,action_df['step']], axis =1)
    step_df['step']= step_df['step'].fillna(100)
    return step_df



def current_position_old(df,last_step):

  df=df.loc[df['step']<last_step+1] ##because starts at 1
  position={'small triangle':set(),'middle triangle':set(),'big triangle':set(),'square':set(),'parallelogram':set()}
  
  for p in df['item'].unique():
    pos= df.loc[df['item']==p].iloc[-1]
    
    #the triangle name does not matter 
    if p == 'big triangle 1' or p == 'big triangle 2':
      p = 'big triangle'
    if p == 'small triangle 1' or p == 'small triangle 2':
      p = 'small triangle'

    position.get(p).add((pos['grid_val'],pos['rot']))
    #position.get(p).append((pos['x'],pos['y'],pos['rot']))
    #position.get(p).append((pos['x'],pos['y']))
  return position

def current_position(df,last_step):

  df=df.loc[:df.loc[(df['step']<last_step+1)].index[-1]] ##because starts at 1
  position={'small triangle':set(),'middle triangle':set(),'big triangle':set(),'square':set(),'parallelogram':set()}
  
  for p in df['item'].unique():
    pos= df.loc[df['item']==p].iloc[-1]
    
    #the triangle name does not matter 
    if p == 'big triangle 1' or p == 'big triangle 2':
      p = 'big triangle'
    if p == 'small triangle 1' or p == 'small triangle 2':
      p = 'small triangle'
    if pos['step'] != 100:
        position.get(p).add((pos['grid_val'],pos['rot']))
    #position.get(p).append((pos['x'],pos['y'],pos['rot']))
    #position.get(p).append((pos['x'],pos['y']))
  return position


In [6]:
#dir_logs = './logs'
dir_logs = './logs_test_ZAL/'
logs =[]

    


for f in os.listdir(dir_logs):
  if f=='old_logs':
    continue
  file = dir_logs + '/' + f
  
  if not "reformated" in file:
    sid = int(f.strip("_log.csv"))
    logs.append(process_log(file, sid)) 
    
moves_df= create_moves_dataframe(logs)
sol_df = create_solution_dataframe(logs)

# states_df #
position of each piece at move 4,8,12,16

In [5]:
step_df = create_grid_step_df(moves_df,5)

['big triangle 1' 'square' 'small triangle 1' 'small triangle 2'
 'big triangle 2' 'middle triangle' 'parallelogram']


In [41]:


puzzles = [1,2,3,4]
players = step_df['sid'].unique()

states_df =pd.DataFrame(columns=['sid','tangram nr','step','small triangle','middle triangle','big triangle','square','parallelogram'])



for player in players:
  if player == 104401:
    continue
  for pzn in puzzles:
    partial  = step_df.loc[(step_df['tangram nr']==pzn) & (step_df['sid']==player)]
    for step in range(1,17):
    #for step in range(1,17):
        pos = current_position(partial,step)
      
        
      #print(pos)
        row = {'sid':player, 'tangram nr': pzn, 'step':step, 'small triangle':pos.get('small triangle'),
                 'middle triangle': pos.get('middle triangle'), 'big triangle' : pos.get('big triangle'),
                 'square': pos.get('square'), 'parallelogram': pos.get('parallelogram')}
        states_df = states_df.append(row, ignore_index =True)

states_df.to_csv('./datasets/train_states.csv')

['big triangle 1' 'square' 'small triangle 1' 'small triangle 2'
 'big triangle 2' 'middle triangle' 'parallelogram']


# Creating the frequency datasets #


In [7]:
grid_df = create_grid_step_df(moves_df,5)
#grid_df = grid_df.loc[grid_df.step != 100]

['big triangle 1' 'big triangle 2' 'parallelogram' 'small triangle 1'
 'small triangle 2' 'middle triangle' 'square']


In [8]:

grid_df['rot']=grid_df.apply(lambda x: x['rot'] % 90 if x['item'] == 'square' else x['rot'], axis =1)
grid_df['rot']=grid_df.apply(lambda x: x['rot'] % 180 if x['item'] == 'parallelogram' else x['rot'], axis =1)
#grid_df['item']=grid_df.apply(lambda x: 'big triangle' if 'big triangle'in x['item']else x['item'], axis =1)
#grid_df['item']=grid_df.apply(lambda x: 'small triangle' if 'small triangle' in x['item'] else x['item'], axis =1)
di = {'small triangle 1':'SMALL-T','small triangle 2':'SMALL-T','middle triangle':'MIDDLE-T',
      'big triangle 1':'BIG-T','big triangle 2':'BIG-T','square':'SQUARE','parallelogram':'PARALL'}
item_n = {'small triangle 1':3,'small triangle 2':4,'middle triangle':2,
      'big triangle 1':0,'big triangle 2':1,'square':5,'parallelogram':6}
grid_df['i_item']=grid_df['item'].apply(lambda x: item_n.get(x) )
grid_df['item']=grid_df['item'].apply(lambda x: di.get(x) )

In [10]:
### landmark counts: number of times a certain action is been taken between step 0 and phase ###
grid_df.to_csv('./datasets/all_steps_test_ZAL.csv')

# for phase in [4,8,12,16]:
#     sums = grid_df.loc[grid_df.step<=phase].groupby(['tangram nr','item','grid_val','rot']).size().reset_index(name='counts')
#     sums.to_csv(f'./datasets/landmark_counts_{phase}.csv')

In [11]:
grid_df = pd.read_csv('./datasets/all_steps_test_ZAL.csv')
grid_df.loc[grid_df['tangram nr'] == 4].head(30)

,Unnamed: 0,sid,tangram nr,item,x,y,rot,grid_val,step,i_item
73,73,91821,4,PARALL,-194.0,91.0,90.0,11.0,1.0,6
74,74,91821,4,BIG-T,-62.0,72.0,270.0,13.0,2.0,0
75,75,91821,4,SQUARE,47.0,-102.0,45.0,-1.0,100.0,5
76,76,91821,4,SMALL-T,-138.0,-151.0,135.0,2.0,3.0,3
77,77,91821,4,BIG-T,-129.0,-46.0,45.0,7.0,4.0,0
78,78,91821,4,BIG-T,-131.0,99.0,135.0,12.0,5.0,1
79,79,91821,4,MIDDLE-T,-114.0,-63.0,225.0,7.0,6.0,2
80,80,91821,4,SQUARE,-65.0,206.0,0.0,23.0,7.0,5
81,81,91821,4,SMALL-T,-65.0,243.0,0.0,23.0,8.0,4
134,134,100011,4,BIG-T,-62.0,67.0,270.0,13.0,100.0,0


# Relative strength dataframe #

Relative strength calculated from the placed pieces at a certain position during the solution phase

In [16]:
step_df = create_grid_step_df(moves_df,5)
grid_df = grid_df.loc[grid_df.step != 100]
pieces = step_df.item.unique()
players = step_df.sid.unique()
last_positions_df = pd.DataFrame(columns=['sid','tangram nr','step','item','grid_val','rot'])
cnt =0
for player in players:
  for pzn in puzzles:
    #for step in [4,8,12,16]:
    for step in range(1,19):
        partial  = step_df.loc[(step_df['tangram nr']==pzn) & (step_df['sid']==player) & (step_df['step']<step+1)]
        
        for p in pieces:
            
            #find last position
            piece_rows= partial.loc[partial['item']==p]
            if p == 'big triangle 1' or p == 'big triangle 2':
              p = 'big triangle'
            if p == 'small triangle 1' or p == 'small triangle 2':
              p = 'small triangle'
            if not piece_rows.empty:
                cnt+=1
                item_pos = piece_rows.iloc[-1]
                piece_row = {'sid':player, 'tangram nr': pzn, 'step': step, 'item':p, 'grid_val': item_pos['grid_val'],
                             'rot': item_pos['rot']}
            else:
                cnt+=1
                piece_row = {'sid':player, 'tangram nr': pzn, 'step': step, 'item':p, 'grid_val': -1,
                             'rot': 0}
            last_positions_df= last_positions_df.append(piece_row, ignore_index = True)


['big triangle 1' 'parallelogram' 'small triangle 1' 'square'
 'small triangle 2' 'middle triangle' 'big triangle 2']


NameError: name 'puzzles' is not defined

In [46]:
### landmark str: (relative) number of times a certain action is been taken between step phase-4 and phase ###

gdf = last_positions_df[~last_positions_df.grid_val.isin([-1])] #need to change wrt puzzlen
gdf['rot']=gdf.apply(lambda x: x['rot'] % 90 if x['item'] == 'square' else x['rot'], axis =1)
gdf['rot']=gdf.apply(lambda x: x['rot'] % 180 if x['item'] == 'parallelogram' else x['rot'], axis =1)
#gdf = last_positions_df.copy()
di = {'small triangle':'SMALL-T','middle triangle':'MIDDLE-T',
      'big triangle':'BIG-T','square':'SQUARE','parallelogram':'PARALL'}
gdf['item']=gdf['item'].apply(lambda x: di.get(x) )
sums=[]

phases = [5,9,13,17]
offsets = [0,1,2,3]

for i in range(len(phases)):
    rsums = pd.DataFrame({'counts':gdf.loc[(gdf['step']>=phases[i]-4) & (gdf['step']<phases[i]+offsets[i])].
                        groupby(['tangram nr','item','grid_val','rot']).size()}).reset_index()
    rsums=rsums.loc[rsums.counts > 3] ### new addition
    rsums['strength'] = rsums.counts/rsums.groupby('tangram nr').counts.transform('sum')
    
    sums.append(rsums)
    rsums.to_csv(f'./datasets/landmark_str_{i}_test.csv')
    


Mainly debug here

In [52]:
df = pd.read_csv('./datasets/landmark_str_0.csv')
df.loc[(df['tangram nr']==2) & (df.item == 'SQUARE')]

,Unnamed: 0,tangram nr,item,grid_val,rot,counts,strength
28,75,2,SQUARE,21.0,0.0,4,0.014235
29,76,2,SQUARE,22.0,0.0,73,0.259786


In [52]:
def get_grid_value_new(rm, cm, n, tgn):
    '''
    Converts x and y matrix coordinates into grid values for tangram puzzle = tgn

    :param rm: row value in matrix coordinates
    :type rm: int
    :param cm: column value in matrix coordinates
    :type cm: int
    :param n: grid size
    :type n: int
    :return:  grid number 0-15, or -1 if outside the grid
    '''
    solution_limits = {1: [(-260, 120), (-120, 140)], 2: [(-280, 0), (-70, 210)], 3: [(-320, 60), (-140, 140)],
                       4: [(-280, 0), (-200, 300)]}
    y = -rm + 300
    x = cm - 400

    xrange = solution_limits.get(tgn)[0]
    yrange = solution_limits.get(tgn)[1]
    xstep = (xrange[1] - xrange[0]) / n
    ystep = (yrange[1] - yrange[0]) / n
    if (x not in range(xrange[0], xrange[1] + 1) or y not in range(yrange[0], yrange[1] + 1)):
        return -1
    xgrid = (x - xrange[0]) // xstep

    ygrid = (y - yrange[0]) // ystep

    return x, y, ygrid * n + xgrid



In [54]:
ap=[((364, 218), 0, 0.021266540642722116), ((364, 313), 0, 0.021739130434782608), ((312, 166), 90, 0.020793950850661626), ((299, 333), 135, 0.028904428904428906), ((332, 231), 225, 0.0), ((297, 196), 225, 0.05920745920745921), ((332, 196), 225, 0.059673659673659674), ((312, 365), 270, 0.021739130434782608)]

In [55]:
[(int(get_grid_value_new(x[0][0],x[0][1],5,2)[2]),x[1]) for x in ap]

[(1, 0), (3, 0), (5, 90), (8, 135), (1, 225), (6, 225), (1, 225), (9, 270)]

# Old code

In [ ]:
def create_grid_df(df):
  step_df= df[df["action"]!='turned 45 degrees right']
  step_df =step_df.loc[(step_df['item'].shift(-1) != step_df['item']) 
                          | (step_df['tangram nr'].shift(-1) != step_df['tangram nr'])
                          ].reset_index(drop=True) 

  step_df['step'] = step_df.groupby((step_df['tangram nr'] != step_df['tangram nr'].shift(1)).cumsum()).cumcount()+1

  pos = step_df['position'].tolist()
  pos=[eval(x) for x in pos]


  step_df[['x','y','rot']] = pd.DataFrame(pos, index=step_df.index)
  step_df = step_df.drop(['position','time','action'], axis=1)

  #fix wrong logs
  diff_logs = [102957,104401,102644,100444,92356,93250]
  #step_df['y'] = step_df.apply(lambda x: x['y']-70 if (x['tangram nr']==4 and x['sid'] in diff_logs)  else x['y'], axis=1)
  step_df['y'] = step_df.apply(lambda x: x['y']-70 if x['sid'] in diff_logs  else x['y'], axis=1)
  step_df['grid_val']=step_df.apply(lambda x: get_grid_value(x['x'],x['y'],x['tangram nr']),axis=1)
  return step_df